# Demo filter

A demonstration of the edge filter with controllable strength.

In [1]:
import numpy as np
import cv2
from scipy.signal import convolve2d

def gaussian_blur(im, ksize):
    # odd kernel size only
    k = max(1, int(ksize))
    if k % 2 == 0:
        k += 1
    return cv2.GaussianBlur(im, (k, k), 0)

cv2.startWindowThread()
camera = cv2.VideoCapture(0)

blur_size = 3  # initial blur level

try:
    while True:
        ret, im = camera.read()
        if not ret:
            print("Failed to grab frame")
            break

        im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

        # Apply blur with current intensity
        im_blur = gaussian_blur(im, blur_size)

        cv2.imshow('Adjustable Blur Filter', im_blur.astype(np.uint8))

        key = cv2.waitKey(1) & 0xFF

        # Increase blur intensity
        if key in [ord('+'), ord('='), 82]:  # 82 = up arrow key on some systems
            blur_size = min(99, blur_size + 2)
            print(f'Blur size: {blur_size}')

        # Decrease blur intensity
        elif key in [ord('-'), ord('_'), 84]:  # 84 = down arrow key
            blur_size = max(1, blur_size - 2)
            print(f'Blur size: {blur_size}')

        # Quit
        elif key == ord('q'):
            break

finally:
    camera.release()
    cv2.destroyAllWindows()
    for _ in range(4):
        cv2.waitKey(1)
